In [2]:
import dns.query
import dns.message
import re
import ipaddress
import time
import datetime
import random

In [3]:
cache = {}

In [4]:
root_servers = {}

root_servers['a'] = '198.41.0.4'
root_servers['b'] = '199.9.14.201'
root_servers['c'] = '192.33.4.12'
root_servers['d'] = '199.7.91.13'
root_servers['e'] = '192.203.230.10'
root_servers['f'] = '192.5.5.241'
root_servers['g'] = '192.112.36.4'
root_servers['h'] = '198.97.190.53'
root_servers['i'] = '192.36.148.17'
root_servers['j'] = '192.58.128.30'
root_servers['k'] = '193.0.14.129'
root_servers['l'] = '199.7.83.42'
root_servers['n'] = '202.12.27.33'

In [21]:
# parameters

hostname = 'www.163.com'
rdtype = 'A'

In [6]:
def find_cache(hostname, rdtype):
    ''' Find whether cache already has the record
    
    Args:
        hostname (str): hostname
        rdtype (str): type A, NS, or MX
    '''
    key = hostname + ' ' + rdtype
    

In [7]:
key = hostname + ' ' + rdtype

In [8]:
cache[key] = '151.101.21.67'

In [218]:
print(cache.get(key))

151.101.21.67


In [49]:
def single_iterate(hostname, rdtype, where, timeout=1):
    ''' A single iterative DNS query
    
    Args:
        hostname (str): host to be queried
        rdtype (str): type A, NS, or MX
        where (str):  IP address of query destination
    
    Return: 
        response (dns.message.Message): the response of a single query
        
    Exception:
        May raise an exception
    '''
    a_query = dns.message.make_query(hostname, rdtype)
    try:
        #print('single iterate: ', hostname, rdtype, where)
        response = dns.query.udp(a_query, where, timeout)
        return response
    except Exception as e:
        raise e  # Let the block who call this function catch the exception

In [10]:
def check_response(response):
    '''Check whether the response has a valid IP address in its ANSWER section.
    
    Args:
        response (dns.message.Message): the response of a single query
    
    Return:
        True or False
    '''
    try:
        ip = get_ip_from_rrset(response.answer[0])
        ipaddress.ip_address(ip)
        return True
    except Exception as e:
        return False            

In [11]:
def dns_resolver(hostname, rdtype):
    for root in root_servers.values():
        response = single_iterate(hostname, rdtype, root)
        if len(response.additional) == 0:
            continue                     # root doesn't have top level domain information
        while(len(response.answer)==0):  # answer section is empty, then keep interating
            for rrset in response.additional:
                next_ip = get_ip_from_rrset(rrset)
                response = single_iterate(hostname, rdtype, next_ip)
                if len(response.answer)!=0 or len(response.additional)!=0: # if exist answer or exist additional, then the iterate is good
                    break  # if both no answer and no additional, then iterate agian using another ip
        if check_response(response): # ip is in the response
            return response
        else:                         # CNAME is in the response
            for rrset in response.answer:
                cname = get_cname_from_rrset(rrset)
                print(cname)
                return dns_resolver(cname, rdtype)

In [89]:
def dns_resolver_2(hostname, rdtype, cnames):
    ''' My DNS resolver
    
    Args:
        hostname (str): target hostname
        rdtype (str):   type A, NS, or MX
        cnames (list):  a list of CNAMES during a dns query
        
    Return:
        response (dns.message.Message): response of this dns query
    '''
    for root in root_servers.values():
        try:
            response = single_iterate(hostname, rdtype, root, timeout=1)
            if len(response.additional) == 0:
                continue                           # root doesn't have top level domain information
            while(len(response.answer)==0 ):       # if ANSWER section is empty, then keep iterating
                if len(response.additional) > 0:   # use the IP in ADDITIONAL section
                    for rrset in response.additional:
                        next_ip = get_ip_from_rrset(rrset)
                        try:
                            response2 = single_iterate(hostname, rdtype, next_ip, timeout=1)
                            response = response2
                            break
                        except Exception as e:
                            pass  # print('Oops! Authoratative server timeout, try next one. ', e)
                else:             # if both ANSWER and ADDITIONAL is empty, then find the IP of AUTHORITY  
                    ns = get_ns_from_authority(response)
                    response2 = dns_resolver_2(ns, rdtype, cnames)
                    authority_answer = response2.answer[0]
                    response.additional.append(authority_answer)  # add rrset that contains IP of a AUTHORITY to response
            if check_response(response):  # ip is in the response
                return response
            else:                         # CNAME is in the response
                for rrset in response.answer:
                    cname = get_cname_from_rrset(rrset)
                    cnames.append(cname)
                    return dns_resolver_2(cname, rdtype, cnames)
            break
        except Exception as e:
            pass   # print('Oops! Some error, start from a new root server.', e)

In [13]:
def get_ns_from_authority(response):
    '''Get a name server from AUTHORITY.
    In some cases, there is no ADDITIONAL section, just AUTHORITY section!
    
    Args:
        response (dns.message.Message)
        
    Return:
        string: A name of ns server
    '''
    rrsets = response.authority[0].items
    index = random.randint(0, len(rrsets)-1)
    return rrsets[index].to_text()

In [88]:
hostname = 'www.163.com'
rdtype = 'A'
cnames = []

start = time.time()

myresponse = dns_resolver_2(hostname, rdtype, cnames)

elapsed = time.time() - start

output(hostname, rdtype, myresponse, elapsed, cnames)

QUESTION SECTION:
www.163.com.                           IN   A    

ANSWER SECTION:
www.163.com.                     60    IN   CNAME  www.163.com.lxdns.com.
www.163.com.lxdns.com.           60    IN   CNAME  opthw.xdwscache.speedcdns.com.
opthw.xdwscache.speedcdns.com.   60    IN   A      203.130.53.126

Query time: 2654 msec
WHEN: Mon Feb 12 16:00:42 2018
MSG SIZE rcvd:  205


In [14]:
def get_cname_from_rrset(rrset):
    '''Get CNAME from a RRset (from ANSWER section)
    
    Args:
        rrset (A DNS RRset): contains an CNAME to be extracted
        
    Returns:
        CNAME (str): the CNAME in the RRset
    '''
    try:
        item = rrset.items[0]
        return item.to_text()
    except Exception as e:
        print('Oops! Some issue with cname: ', e)

In [17]:
def get_ip_from_rrset(rrset):
    ''' Get IP address from a RRset (from ADDITIONAL section)
    
    Args:
        rrset (A DNS RRset ): contains an IP address to be extracted
        
    Return:
        ip (str): the IP address in the RRset
    '''
    try:
        item = rrset.items[0]
        return item.to_text()
    except Exception as e:
        print('Oops! Some issue with ip: ', e)

In [87]:
def output(hostname, rdtype, myresponse, elapsed, cnames):
    '''The output of the program
    
    Args:
        hostname (str): host to be queried
        rdtype (str): type A, NS, or MX
        myresponse (dns.message.Message): reponse from the DNS query
        elapsed (float): time elapsed
        cnames (list): cnames during a dns query
    '''
    answers = []
    for rrset in myresponse.answer:
        for item in rrset.items:
            answers.append(item.to_text())
            
    hostname_ = hostname + '.'
    
    number = 30
    re_number = '(\d+)(.*)' + answers[0]
    match = re.search(re_number, myresponse.to_text())
    if match:
        number = match.group(1)
    number = str(number)
    
    
    first_line = 'QUESTION SECTION:\n'
    second_line = hostname_.ljust(39) + 'IN'.ljust(5) + rdtype.ljust(5) + '\n\n'
    third_line = 'ANSWER SECTION:\n'
    forth_line = ''
    
    if len(cnames) > 0:
        left = hostname_
        for cname in cnames:
            forth_line = forth_line + left.ljust(33) + number.ljust(6) + 'IN'.ljust(5) + 'CNAME'.ljust(7) + cname + '\n'
            left = cname
        for answer in answers:
            forth_line = forth_line + left.ljust(33) + number.ljust(6) + 'IN'.ljust(5) + rdtype.ljust(7) + answer + '\n'
    else:    
        for answer in answers:
            forth_line = forth_line + hostname_.ljust(33) + number.ljust(6) + 'IN'.ljust(5) + rdtype.ljust(7) + answer + '\n'
    
    string = first_line + second_line + third_line + forth_line
    print(string)
    msg_size = len(string.replace(' ', ''))
    print('Query time: ' + str(int(elapsed * 1000)) + ' msec')
    print('WHEN:', datetime.datetime.now().strftime("%a %b %d %H:%M:%S %Y"))
    print('MSG SIZE rcvd: ', msg_size)

In [69]:
hostname = 'www.google.co.jp'
rdtype = 'A'
cnames = []

start = time.time()

myresponse = dns_resolver_2(hostname, rdtype, cnames)

elapsed = time.time() - start

output(hostname, rdtype, myresponse, elapsed, cnames)

Oops! Authoratative server timeout, try next one.  The DNS operation timed out.
Oops! Authoratative server timeout, try next one.  [Errno 65] No route to host
Oops! Authoratative server timeout, try next one.  [Errno 65] No route to host
ns3.google.net. 21600 IN A 216.239.36.10
QUESTION SECTION:
www.google.co.jp.             IN   A    

ANSWER SECTION:
www.google.co.jp.       300   IN   A      173.194.206.94

Query time: 4063 msec
WHEN: Mon Feb 12 15:52:15 2018
MSG SIZE rcvd:  92
